In [1]:
import pandas as pd
import numpy as np
import os; os.chdir('../')
import warnings

import orca
import urbansim_templates

from scripts import models, datasources, variables

warnings.simplefilter('ignore')

Registering model step 'auto_ownership'
Registering model step 'TOD_choice'
Registering model step 'primary_mode_choice'
Registering model step 'WLCM'


### Generate accessibility variables (or load them if they already exist)

Initialize networks

In [2]:
orca.run(['initialize_network_small', 'initialize_network_walk'])

Running step 'initialize_network_small'
Time to execute step 'initialize_network_small': 0.00 s
Running step 'initialize_network_walk'
Time to execute step 'initialize_network_walk': 0.00 s
Total time to execute iteration 1 with iteration value None: 0.00 s


If they exist already:

In [3]:
walk_net_vars = pd.read_csv('../ual_model_workspace/spring-2019-models/data/walk_net_vars.csv', index_col='osmid')
drive_net_vars = pd.read_csv('../ual_model_workspace/spring-2019-models/data/drive_net_vars.csv', index_col='osmid')
orca.add_table('nodeswalk', walk_net_vars)
orca.add_table('nodessmall', drive_net_vars)

If generating on-the-fly:

In [4]:
# orca.run(['network_aggregations_small', 'network_aggregations_walk'])

### Simulate workplace location choices

Run the model step

In [5]:
orca.run(['wlcm_simulate'])

Running step 'wlcm_simulate'
Iteration 1: 191168 of 3060996 valid choices
Iteration 2: 381577 of 3060996 valid choices
Iteration 3: 571196 of 3060996 valid choices
Iteration 4: 760222 of 3060996 valid choices
Iteration 5: 947787 of 3060996 valid choices
Iteration 6: 1133998 of 3060996 valid choices
Iteration 7: 1318301 of 3060996 valid choices
Iteration 8: 1500725 of 3060996 valid choices
Iteration 9: 1680176 of 3060996 valid choices
Iteration 10: 1855866 of 3060996 valid choices
Iteration 11: 2026046 of 3060996 valid choices
Iteration 12: 2187810 of 3060996 valid choices
Iteration 13: 2335721 of 3060996 valid choices
Iteration 14: 2459545 of 3060996 valid choices
Iteration 15: 2540550 of 3060996 valid choices
Iteration 16: 2573867 of 3060996 valid choices
Iteration 17: 2578046 of 3060996 valid choices
No valid alternatives for the remaining choosers
Time to execute step 'wlcm_simulate': 229.92 s
Total time to execute iteration 1 with iteration value None: 229.92 s


Query the updated persons table to assess job placement

In [6]:
persons_w_jobs = orca.get_table('persons').to_frame()
num_persons_w_jobs = len(persons_w_jobs[~pd.isnull(persons_w_jobs['job_id'])])
wlcm = urbansim_templates.modelmanager.get_step('WLCM')
num_workers = len(persons_w_jobs.query(' and '.join(wlcm.out_chooser_filters)))

In [7]:
print("WLCM placed {0} of {1} workers ({2}%) during simulation.".format(
    num_persons_w_jobs, num_workers, np.round(num_persons_w_jobs/num_workers * 100, 1)))

WLCM placed 2578046 of 3060996 workers (84.2%) during simulation.


### Simulate time-of-day choices for mandatory trip (commute) departure times

#### a) Assign commuters to depature time chunks

In [8]:
orca.run(['TOD_choice_simulate'])

Running step 'TOD_choice_simulate'
Time to execute step 'TOD_choice_simulate': 218.18 s
Total time to execute iteration 1 with iteration value None: 218.18 s


#### b) Assign actual departure times based on known distributions

In [9]:
orca.run(['TOD_distribution_simulate'])

Running step 'TOD_distribution_simulate'
Time to execute step 'TOD_distribution_simulate': 622.52 s
Total time to execute iteration 1 with iteration value None: 622.52 s


Query the updated persons table to assess TOD chunk assignment

In [10]:
persons_w_TOD2 = orca.get_table('persons').to_frame()
num_persons_w_TOD = len(persons_w_TOD2[~pd.isnull(persons_w_TOD2['HW_ST'])])

In [11]:
print("Departure time assigned to {0} of {1} workers with jobs ({2}%) during simulation.".format(
    num_persons_w_TOD, num_persons_w_jobs, np.round(num_persons_w_TOD/num_persons_w_jobs * 100, 1)))

Departure time assigned to 2502285 of 2578046 workers with jobs (97.1%) during simulation.


### Simulate auto-ownership choices

In [12]:
orca.run(['auto_ownership_simulate'])

Running step 'auto_ownership_simulate'
Time to execute step 'auto_ownership_simulate': 272.90 s
Total time to execute iteration 1 with iteration value None: 272.90 s


In [13]:
households_w_auto_ownership = orca.get_table('households').to_frame()
num_households_w_auto = len(households_w_auto_ownership[~pd.isnull(households_w_auto_ownership['cars'])])

In [14]:
print("Auto ownership assigned to {0} of {1} households ({2}%) during simulation.".format(
    num_households_w_auto, len(households_w_auto_ownership),
    np.round(num_households_w_auto/len(households_w_auto_ownership) * 100, 1)))

Auto ownership assigned to 2677468 of 2677468 households (100.0%) during simulation.


### Simulate primary commute mode choice

In [15]:
orca.run(['primary_mode_choice_simulate'])

Running step 'primary_mode_choice_simulate'
Time to execute step 'primary_mode_choice_simulate': 192.55 s
Total time to execute iteration 1 with iteration value None: 192.55 s


In [16]:
persons_w_mode = orca.get_table('persons_CHTS_format').to_frame()

In [17]:
num_persons_w_mode = len(persons_w_mode[~pd.isnull(persons_w_mode['primary_commute_mode'])])

In [18]:
print("Primary commute mode assigned to {0} of {1} workers with jobs ({2}%) during simulation.".format(
    num_persons_w_mode, num_persons_w_jobs,
    np.round(num_persons_w_mode/num_persons_w_jobs * 100, 1)))

Primary commute mode assigned to 2578046 of 2578046 workers with jobs (100.0%) during simulation.


### Generate activity plans

Activity plans currently written according to [BEAM](http://beam.lbl.gov/) specification

In [19]:
orca.run(['generate_activity_plans'], spec='BEAM')

Running step 'generate_activity_plans'
Time to execute step 'generate_activity_plans': 207.92 s
Total time to execute iteration 1 with iteration value None: 207.92 s


Load activity plan data and investigate it

In [20]:
activities = pd.read_csv('./data/urbansim_beam_plans.csv')

In [21]:
activities.head()

,personId,planElement,planElementIndex,activityType,x,y,endTime
0,0,activity,1,Home,-121.882277,37.339126,6.598267
1,0,leg,2,NaN,NaN,NaN,NaN
2,0,activity,3,Work,-121.759406,37.666030,17.413735
3,0,leg,4,NaN,NaN,NaN,NaN
4,0,activity,5,Home,-121.882277,37.339126,NaN


In [22]:
num_persons_w_plans = len(activities['personId'].unique())

In [23]:
print("Mandatory trips assigned to {0} of {1} workers with jobs ({2}%) during simulation.".format(
    num_persons_w_plans, num_persons_w_jobs,
    np.round(num_persons_w_mode/num_persons_w_jobs * 100, 1)))

Mandatory trips assigned to 2578046 of 2578046 workers with jobs (100.0%) during simulation.


### Archive input data

In [35]:
orca.write_tables('./data/urbansim_beam_input_data.h5', [
    'persons', 'households', 'jobs', 'parcels', 'buildings', 'units'
])